In [8]:
# -*- coding: utf-8 -*-
"""
FACTORISATION DE POLYNÔMES DANS Z[X]
Algorithme du grand nombre premier avec remontée (Hensel)
Version pédagogique avec commentaires détaillés
"""

# Importation des modules nécessaires
from sage.all import *          # Importe toutes les fonctions de SageMath
from itertools import combinations  # Pour générer des combinaisons de facteurs

def factoriser_polynome(P):
    """
    Fonction principale qui factorise un polynôme dans Z[X]
    
    Paramètres:
        P -- Un polynôme à coefficients entiers (de type Polynomial_integer_dense)
    
    Retourne:
        - Un facteur non trivial si P est réductible
        -P si P est irréductible
    
    """
    
    ###########################################################################
    # ÉTAPE 1: PRÉPARATION DU POLYNÔME
    ###########################################################################
    
    # On récupère l'anneau parent Z[x] du polynôme P
    Zx = P.parent()  
    
    # Vérification et normalisation du polynôme:
    # 1. On rend le polynôme primitif (PGCD des coeffs = 1)
    if not P.is_primitive():
        # Le contenu est le PGCD des coefficients
        contenu = P.content()
        # On divise le polynôme par son contenu
        P = P / contenu
    
    # 2. On élimine les facteurs carrés éventuels
    # (nécessaire pour la méthode de factorisation)
    P = P.squarefree_part()
    
    ###########################################################################
    # ÉTAPE 2: CALCUL DE LA BORNE DE MIGNOTTE
    ###########################################################################
    
    # La borne de Mignotte permet de majorer les coefficients des facteurs
    
    n = P.degree()  # Degré du polynôme
    
    # Calcul de la norme infinie ||P||∞ = max(|coeffs|)
    C = max(abs(coeff) for coeff in P.coefficients())
    
    # Formule exacte de la borne de Mignotte:
    B = sqrt(n + 1) * (2**n) * C
    
    # Conversion en nombre flottant pour les comparaisons
    B = B.n()  
    
    ###########################################################################
    # ÉTAPE 3: CHOIX DU NOMBRE PREMIER
    ###########################################################################
    
    # On choisit un nombre premier p > 2B
    p = next_prime(2 * ceil(B))
    
    # Boucle pour trouver un bon premier p:
    # - p doit être > 2B
    # - P doit rester sans facteur carré modulo p
    while True:
        try:
            # Réduction du polynôme modulo p
            Pp = P.change_ring(GF(p))
            
            # Vérification que P est bien sans facteur carré modulo p
            if Pp.is_squarefree():
                break  # On a trouvé notre p
                
            # Sinon, on prend le premier suivant
            p = next_prime(p)
            
        except:
            # En cas d'erreur, on prend le premier suivant
            p = next_prime(p)
    
    ###########################################################################
    # ÉTAPE 4: FACTORISATION MODULO p
    ###########################################################################
    
    # Factorisation dans Z/pZ[X]
    decomposition_modp = Pp.factor()
    
    # Extraction des facteurs unitaires
    facteurs_modp = [f[0] for f in decomposition_modp]
    
    # Si un seul facteur, le polynôme est irréductible
    if len(facteurs_modp) == 1:
        return None
    
    ###########################################################################
    # ÉTAPE 5: REMONTÉE DANS Z[X]
    ###########################################################################
    
    # Conversion des facteurs mod p en polynômes dans Z[X]
    facteurs_Z = []
    for f in facteurs_modp:
        # On relève les coefficients dans l'intervalle [-p/2, p/2]
        f_Z = Zx([coeff.lift_centered() for coeff in f])
        facteurs_Z.append(f_Z)
    
    ###########################################################################
    # ÉTAPE 6: RECHERCHE DES FACTEURS DANS Z[X]
    ###########################################################################
    
    # On teste toutes les combinaisons possibles de facteurs
    for r in range(1, len(facteurs_Z)):
        # Génère toutes les combinaisons de r facteurs
        for indices in combinations(range(len(facteurs_Z)), r):
            # Calcul du produit des facteurs
            produit = prod([facteurs_Z[i] for i in indices])
            
            # Test de divisibilité dans Z[X]
            if P % produit == 0:
                return produit  # Facteur trouvé!
    
    # Théoriquement, ce cas ne devrait pas se produire
    return None

##############################################################################
# EXEMPLE D'UTILISATION
##############################################################################

if __name__ == "__main__":
    print("=== FACTORISATION DE POLYNÔMES ===")
    
    # Définition de l'anneau Z[x]
    Zx.<x> = ZZ[]
    
    # Exemple 1: Polynôme réductible
    P1 = x^4 + 2*x^3 + 3*x^2 + 4*x + 2
    print("\nPolynôme à factoriser P1 =", P1)
    
    facteur = factoriser_polynome(P1)
    if facteur:
        print(f"Un facteur non trivial de P1 est: {facteur}")
        print(f"On peut donc écrire P1 = {facteur} * Q1(x)")
    else:
        print("Le polynôme P1 est irréductible dans Z[X]")
        
    # Exemple 2: Polynôme irréductible
    
    P2 = x^3 + x + 1
    print("\nPolynôme à factoriser P2 =", P2)
    
    facteur = factoriser_polynome(P2)
    if facteur:
        print(f"Un facteur non trivial de P2 est: {facteur}")
        print(f"On peut donc écrire P2 = {facteur} * Q2(x)")
    else:
        print("Le polynôme P2 est irréductible dans Z[X]")
        
    # Exemple P3: Polynôme réductible bien visible
    P3 = x^6 - 1  # Se factorise explicitement
    print("\nPolynôme à factoriser P3 =", P3)
    
    facteur = factoriser_polynome(P3)
    if facteur:
        print(f"Un facteur non trivial de P3 est: {facteur}")
        print(f"On peut donc écrire P3 = {facteur} * Q3(x)")
    else:
        print("Le polynôme P3 est irréductible dans Z[X]")


=== FACTORISATION DE POLYNÔMES ===

Polynôme à factoriser P1 = x^4 + 2*x^3 + 3*x^2 + 4*x + 2
Le polynôme P1 est irréductible dans Z[X]

Polynôme à factoriser P2 = x^3 + x + 1
Le polynôme P2 est irréductible dans Z[X]

Polynôme à factoriser P3 = x^6 - 1
Un facteur non trivial de P3 est: x + 1
On peut donc écrire P3 = x + 1 * Q3(x)
